# 🧪 Prédiction de fertilisants – Kaggle Challenge
**Auteur** : Arnaud Chéridi
📅 Date : Juin 2025
📍 Compétition : [Playground Series - Season 3, Episode 16](https://www.kaggle.com/competitions/playground-series-s3e16)
🎯 Objectif : Prédire les 3 fertilisants les plus probables à partir de données agronomiques synthétiques.

---

# Modélisation

Ce notebook contient les étapes de sélection de variables, d’entraînement de plusieurs modèles de machine learning, ainsi que leur évaluation selon la métrique de la compétition.

In [8]:
import pandas as pd
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.ensemble import StackingClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [5]:
train = pd.read_csv('data/train.csv')

## Prétraitement des données

In [12]:
categorical_features = ['Soil Type', 'Crop Type']

label_encoder = LabelEncoder()
train['Fertilizer Encoded'] = label_encoder.fit_transform(train['Fertilizer Name'])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(), categorical_features),
        ('num', StandardScaler(), ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous'])
    ],
    remainder='drop'
)

## Mise en place des grilles de recherche

In [ ]:
model_grid = {
    "LogisticRegression": (
        LogisticRegression(max_iter=1000),
        {"classifier__C": [0.01, 0.1, 1, 10, 100], "classifier__penalty": ['l2'], "classifier__solver": ['lbfgs']}
    ),
    "KNN": (
        KNeighborsClassifier(),
        {"classifier__n_neighbors": [3, 5, 7, 9], "classifier__weights": ['uniform', 'distance']}
    ),
    "DecisionTree": (
        DecisionTreeClassifier(random_state=42),
        {"classifier__max_depth": [None, 10, 20, 30], "classifier__min_samples_split": [2, 5, 10]}
    ),
    "NaiveBayes": (
        GaussianNB(),
        {}
    ),
    "RandomForest": (
        RandomForestClassifier(random_state=42),
        {"classifier__n_estimators": [100, 200, 300], "classifier__max_depth": [None, 10, 20], "classifier__min_samples_split": [2, 5]}
    ),
    "GradientBoosting": (
        GradientBoostingClassifier(),
        {"classifier__n_estimators": [100, 200], "classifier__learning_rate": [0.1, 0.05], "classifier__max_depth": [3, 5]}
    ),
    "HistGradientBoosting": (
        HistGradientBoostingClassifier(),
        {"classifier__learning_rate": [0.1, 0.05], "classifier__max_iter": [100, 200], "classifier__max_depth": [None, 10]}
    ),
    "MLP": (
        MLPClassifier(max_iter=500),
        {"classifier__hidden_layer_sizes": [(100,), (50, 50), (100, 50)], "classifier__alpha": [0.0001, 0.001, 0.01]}
    ),
    "XGBoost": (
        XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
        {"classifier__n_estimators": [100, 200], "classifier__learning_rate": [0.1, 0.05], "classifier__max_depth": [3, 6]}
    ),
    "LightGBM": (
        LGBMClassifier(),
        {"classifier__n_estimators": [100, 200], "classifier__learning_rate": [0.1, 0.05], "classifier__num_leaves": [31, 50]}
    ),
    "CatBoost": (
        CatBoostClassifier(verbose=0),
        {"classifier__iterations": [100, 200], "classifier__learning_rate": [0.1, 0.05], "classifier__depth": [6, 10]}
    )
}

X = train.drop(columns=['Fertilizer Name', 'Fertilizer Encoded', 'id'])
y = train['Fertilizer Encoded']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

results_summary = {}

for name, (model, params) in model_grid.items():
    print(f"\nTesting {name}...")
    pipeline = Pipeline(steps=[
        ('preprocessing', preprocessor),
        ('classifier', model)
    ])
    grid = GridSearchCV(pipeline, param_grid=params, cv=5, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train, y_train)

    y_pred = grid.predict(X_val)
    print(f"Best params: {grid.best_params_}")
    acc = accuracy_score(y_val, y_pred)
    print(f"Accuracy: {acc:.4f}")
    results_summary[name] = {
        "model": grid.best_estimator_,
        "best_params": grid.best_params_,
        "accuracy": acc
    }

    print(classification_report(y_val, y_pred, target_names=label_encoder.classes_))

In [ ]:
with open('results_summary.pkl', 'wb') as file:
    pickle.dump(results_summary, file)

In [ ]:
with open('label_y.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

In [9]:
with open('results_summary.pkl', 'rb') as file:
    results_summary = pickle.load(file)

with open('label_y.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

## Entrainement d'un modèle de stacking avec les quatres meilleures modèles

In [10]:
top_4 = sorted(results_summary.items(), key=lambda x: x[1]["accuracy"], reverse=True)[:4]

In [13]:
X = train.drop(columns=['Fertilizer Name', 'Fertilizer Encoded', 'id'])
y = train['Fertilizer Encoded']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
estimators = [(name.lower(), model_info["model"].named_steps["classifier"]) for name, model_info in top_4]

stacking = StackingClassifier(
    estimators=estimators,
    final_estimator=XGBClassifier(),
    cv=5,
    n_jobs=-1
)
param_grid = {
    'classifier__final_estimator__learning_rate': [0.1, 0.05],
    'classifier__final_estimator__n_estimators': [100, 200],
    'classifier__final_estimator__max_depth': [3, 5]
}

pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', stacking)
])

grid = GridSearchCV(pipeline, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid.fit(X, y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004112 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 500000, number of used features: 8
[LightGBM] [Info] Start training from score -1.884862
[LightGBM] [Info] Start training from score -1.880066
[LightGBM] [Info] Start training from score -1.897533
[LightGBM] [Info] Start training from score -1.911544
[LightGBM] [Info] Start training from score -1.909125
[LightGBM] [Info] Start training from score -2.067671
[LightGBM] [Info] Start training from score -2.094839
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018629 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 500000, number of

[23:25:06] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1742444258230/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[23:25:07] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1742444258230/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[23:25:07] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1742444258230/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.047906 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 500000, number of used features: 8
[LightGBM] [Info] Start training from score -1.884862
[LightGBM] [Info] Start training from score -1.880053
[LightGBM] [Info] Start training from score -1.897547
[LightGBM] [Info] Start training from score -1.911544
[LightGBM] [Info] Start training from score -1.909112
[LightGBM] [Info] Start training from score -2.067671
[LightGBM] [Info] Start training from score -2.094856
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009608 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 500000, number of

[23:25:07] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1742444258230/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

[23:25:07] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1742444258230/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.



[LightGBM] [Info] Start training from score -1.884862
[LightGBM] [Info] Start training from score -1.880066
[LightGBM] [Info] Start training from score -1.897533
[LightGBM] [Info] Start training from score -1.911544
[LightGBM] [Info] Start training from score -1.909125
[LightGBM] [Info] Start training from score -2.067671
[LightGBM] [Info] Start training from score -2.094839
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001957 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 500000, number of used features: 8
[LightGBM] [Info] Start training from score -1.884875
[LightGBM] [Info] Start training from score -1.880053
[LightGBM] [Info] Start training from score -1.897533
[LightGBM] [Info] Start training from score -1.911544
[LightGBM] [Info] Start training from score -1.909125


...

/opt/anaconda3/envs/Data_Science/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036373 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 8
[LightGBM] [Info] Start training from score -1.884862
[LightGBM] [Info] Start training from score -1.880066
[LightGBM] [Info] Start training from score -1.897520
[LightGBM] [Info] Start training from score -1.911557
[LightGBM] [Info] Start training from score -1.909125
[LightGBM] [Info] Start training from score -2.067671
[LightGBM] [Info] Start training from score -2.094839
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001730 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 400000, number of

/opt/anaconda3/envs/Data_Science/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/Data_Science/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001685 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 8
[LightGBM] [Info] Start training from score -1.884879
[LightGBM] [Info] Start training from score -1.880050
[LightGBM] [Info] Start training from score -1.897537
[LightGBM] [Info] Start training from score -1.911540
[LightGBM] [Info] Start training from score -1.909125
[LightGBM] [Info] Start training from score -2.067671
[LightGBM] [Info] Start training from score -2.094839
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.170040 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 400000, number of

/opt/anaconda3/envs/Data_Science/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/Data_Science/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/Data_Science/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/Data_Science/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/Data_Science/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LG